In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from numpy import concatenate
from math import sqrt


In [2]:
df = pd.read_csv('/Users/alexellard/Downloads/scenarios_data.csv', header=0, index_col=0)
df

,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,...,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-3348.0,154.8,169.2,18.0,118.8,169.2,234.0,86.4,82.8,104.4,...,439.2,-205.2,-75.6,-18.0,118.8,68.4,7.2,-54.0,75.6,212.4
2017-01-01 00:30:00,-2959.2,136.8,147.6,18.0,108.0,144.0,237.6,79.2,75.6,90.0,...,392.4,-187.2,-72.0,-21.6,108.0,61.2,3.6,-54.0,72.0,183.6
2017-01-01 01:00:00,-2692.8,129.6,126.0,14.4,104.4,133.2,205.2,72.0,72.0,90.0,...,349.2,-176.4,-64.8,-18.0,93.6,50.4,3.6,-39.6,54.0,151.2
2017-01-01 01:30:00,-2379.6,104.4,100.8,10.8,86.4,122.4,180.0,68.4,57.6,82.8,...,316.8,-158.4,-54.0,-10.8,79.2,46.8,7.2,-32.4,50.4,140.4
2017-01-01 02:00:00,-2228.4,104.4,104.4,10.8,79.2,122.4,183.6,68.4,54.0,75.6,...,277.2,-144.0,-57.6,-18.0,72.0,39.6,7.2,-32.4,43.2,118.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 21:30:00,-5306.4,205.2,208.8,39.6,183.6,284.4,316.8,176.4,133.2,129.6,...,604.8,-259.2,32.4,115.2,169.2,100.8,10.8,-100.8,122.4,309.6
2017-12-31 22:00:00,-4910.4,208.8,183.6,39.6,154.8,252.0,295.2,165.6,133.2,133.2,...,565.2,-241.2,21.6,100.8,154.8,86.4,18.0,-75.6,100.8,288.0
2017-12-31 22:30:00,-4683.6,187.2,183.6,36.0,147.6,230.4,320.4,147.6,118.8,126.0,...,529.2,-226.8,21.6,100.8,147.6,86.4,14.4,-86.4,108.0,266.4


In [3]:
# convert series to supervised learning
def series_to_supervised(data, n_in, n_out, vars_to_predict=None):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i-1))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(n_out):
        cols.append(df[vars_to_predict].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in vars_to_predict]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in vars_to_predict]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    return agg

values = df.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# node 1 indexes
target_variable_demand_node1 = [1]
target_variable_pressures_node1 = [33]
target_variable_flows_node1 = [64]
#node 16 indexes
target_variable_demand_node16 = [16]
target_variable_pressures_node16 = [48]
target_variable_flows_node16 = [79]
#node 29 indexes
target_variable_demand_node29 = [29]
target_variable_pressures_node29 = [61]
target_variable_flows_node29 = [92]


reframed_demand_node1 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_demand_node1)
reframed_pressures_node1 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_pressures_node1)
reframed_flows_node1 = series_to_supervised(scaled, 1, 1, vars_to_predict=target_variable_flows_node1)

reframed_demand_node1 = reframed_demand_node1.drop(reframed_demand_node1.columns[1], axis=1)
reframed_pressures_node1 = reframed_pressures_node1.drop(reframed_pressures_node1.columns[33], axis=1)
reframed_flows_node1 = reframed_flows_node1.drop(reframed_flows_node1.columns[64], axis=1)

In [6]:
# split into train and test sets
demand_values = reframed_demand_node1.values
train, test = train_test_split(demand_values, test_size=0.2, random_state=0)

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# Initialize the Support Vector Regression model
model = SVR(kernel='rbf', C=20, gamma=0.1, epsilon=.1)

# Fit the model on the training data
model.fit(train_X, train_y)

# Use the trained model to make predictions on the test data
y_pred = model.predict(test_X)

# calculate the mean squared error
mae = mean_absolute_error(test_y, y_pred)
print('Mean Absolute Error:', mae)
rmse = sqrt(mean_squared_error(test_y, y_pred))
print('Test RMSE: %.3f' % rmse)
r2 = r2_score(test_y, y_pred)
print('Test R^2: %.3f' % r2)

test_y = test_y.reshape((len(test_y), 1))
inv_test_y = concatenate((test_y, test_X[:, -96:]), axis=1)
test_y_inverse = scaler.inverse_transform(inv_test_y)

y_pred = y_pred.reshape((len(y_pred), 1))
inv_y_pred = concatenate((y_pred, test_X[:, -96:]), axis=1)
y_pred_inverse = scaler.inverse_transform(inv_y_pred)

# evaluate scaled errors
mae_inverse = mean_absolute_error(test_y_inverse, y_pred_inverse)
print('Inverted Mean Absolute Error:', mae_inverse)
rmse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
print('Inverted Test RMSE: %.3f' % rmse)
r2_inverse = r2_score(test_y_inverse, y_pred_inverse)
print('Inverted Test R^2: %.3f' % r2)

Mean Absolute Error: 0.025266061271932372
Test RMSE: 0.032
Test R^2: 0.963
Inverted Mean Absolute Error: 2.2455316394214426
Inverted Test RMSE: 27.496
Inverted Test R^2: 0.963


In [7]:
# split into train and test sets
pressure_values = reframed_pressures_node1.values
train, test = train_test_split(pressure_values, test_size=0.2, random_state=0)

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# Initialize the Support Vector Regression model
model = SVR(kernel='rbf', C=20, gamma=0.1, epsilon=.1)

# Fit the model on the training data
model.fit(train_X, train_y)

# Use the trained model to make predictions on the test data
y_pred = model.predict(test_X)

# calculate the mean squared error
mae = mean_absolute_error(test_y, y_pred)
print('Mean Absolute Error:', mae)
rmse = sqrt(mean_squared_error(test_y, y_pred))
print('Test RMSE: %.3f' % rmse)
r2 = r2_score(test_y, y_pred)
print('Test R^2: %.3f' % r2)

test_y = test_y.reshape((len(test_y), 1))
inv_test_y = concatenate((test_y, test_X[:, -96:]), axis=1)
test_y_inverse = scaler.inverse_transform(inv_test_y)

y_pred = y_pred.reshape((len(y_pred), 1))
inv_y_pred = concatenate((y_pred, test_X[:, -96:]), axis=1)
y_pred_inverse = scaler.inverse_transform(inv_y_pred)

# evaluate scaled errors
mae_inverse = mean_absolute_error(test_y_inverse, y_pred_inverse)
print('Inverted Mean Absolute Error:', mae_inverse)
rmse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
print('Inverted Test RMSE: %.3f' % rmse)
r2_inverse = r2_score(test_y_inverse, y_pred_inverse)
print('Inverted Test R^2: %.3f' % r2)

Mean Absolute Error: 0.06382895420184397
Test RMSE: 0.069
Test R^2: 0.760
Inverted Mean Absolute Error: 5.672809121927063
Inverted Test RMSE: 59.750
Inverted Test R^2: 0.760


In [8]:
# split into train and test sets
flow_values = reframed_flows_node1.values
train, test = train_test_split(flow_values, test_size=0.2, random_state=0)

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# Initialize the Support Vector Regression model
model = SVR(kernel='rbf', C=20, gamma=0.1, epsilon=.1)

# Fit the model on the training data
model.fit(train_X, train_y)

# Use the trained model to make predictions on the test data
y_pred = model.predict(test_X)

# calculate the mean squared error
mae = mean_absolute_error(test_y, y_pred)
print('Mean Absolute Error:', mae)
rmse = sqrt(mean_squared_error(test_y, y_pred))
print('Test RMSE: %.3f' % rmse)
r2 = r2_score(test_y, y_pred)
print('Test R^2: %.3f' % r2)

test_y = test_y.reshape((len(test_y), 1))
inv_test_y = concatenate((test_y, test_X[:, -96:]), axis=1)
test_y_inverse = scaler.inverse_transform(inv_test_y)

y_pred = y_pred.reshape((len(y_pred), 1))
inv_y_pred = concatenate((y_pred, test_X[:, -96:]), axis=1)
y_pred_inverse = scaler.inverse_transform(inv_y_pred)

# evaluate scaled errors
mae_inverse = mean_absolute_error(test_y_inverse, y_pred_inverse)
print('Inverted Mean Absolute Error:', mae_inverse)
rmse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
print('Inverted Test RMSE: %.3f' % rmse)
r2_inverse = r2_score(test_y_inverse, y_pred_inverse)
print('Inverted Test R^2: %.3f' % r2)

Mean Absolute Error: 0.022627579095495374
Test RMSE: 0.031
Test R^2: 0.975
Inverted Mean Absolute Error: 2.011038869426667
Inverted Test RMSE: 27.415
Inverted Test R^2: 0.975
